In [1]:
import pandas as pd
import yfinance as yf
from yahoofinancials import YahooFinancials
from datetime import date,datetime, timedelta
import os
import tweepy as tw
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import Cursor
from tweepy import API
from itertools import tee
from get_all_tickers import get_tickers as gt
import time
import re
from textblob import TextBlob

#### Variables setting

In [2]:
olympus = gt.get_tickers(NASDAQ = False, AMEX = False) #all NYSE ticks
#You will need to be sure that you are not targeting more than 2,000 tickers, because the Yfinance API has a 2,000 API calls per hour limit.

ParserError: Error tokenizing data. C error: Expected 1 fields in line 23, saw 46


In [5]:
### Retrieving company information
olympus_details = ["shortName","sector","longBusinessSummary","twoHundredDayAverage","marketCap"] #more available for further analysis
olympus_info = []
counter = 0
for tick in olympus[0:5]: #remove to get for all
    olympus_info.append([yf.Ticker(tick).info[x] for x in olympus_details])
    olympus_info[counter].insert(0, tick)
    counter += 1
    time.sleep(0.01)

NameError: name 'olympus' is not defined

In [7]:
olympus_pd = pd.DataFrame(olympus_info,columns=["Ticker",'Name',"Sector","Description","Two_Hundred_Day_Average", "Market_Cap"])
olympus_pd.head()

,Ticker,Name,Sector,Description,Two_Hundred_Day_Average,Market_Cap
0,DDD,3D Systems Corporation,Technology,"3D Systems Corporation, through its subsidiari...",6.605429,1298525312
1,MMM,3M Company,Industrials,"3M Company develops, manufactures, and markets...",162.164500,99478732800
2,WBAI,500.com Limited,Consumer Cyclical,"500.com Limited, through its subsidiaries, pro...",3.346786,116964080
3,EGHT,8x8 Inc,Technology,"8x8, Inc. provides voice, video, chat, contact...",16.411072,2218919424
4,AHC,A.H. Belo Corporation (TX),Communication Services,"A.H. Belo Corporation, together with its subsi...",1.550643,31901496


In [6]:
### Setting up date period for download
today = date.today()
start = today - timedelta(days=7)

In [7]:
### Downloading stock prices for olympus stocks
athena = yf.download(olympus[0:100], 
                      start=start, 
                      end=today,
                      interval="60m",
                      progress=False)
athena['Close'].head()

NameError: name 'olympus' is not defined

In [ ]:
athena['Close'].plot(title="Olympus")

In [64]:
search_words = " OR ".join(olympus_pd["Name"])
print(search_words[0:23])

3D Systems Corporation 


#### Twitter Download

In [4]:
consumer_key = 'tjb7Mnb9mixcWAOekbRSfe35G'
consumer_secret = 'IQqp3M9mKGV53z8KKQh0pEjf4f0DnBnXZpH8qtST9T6BeTNOlh'
access_token = '2895232949-Jtj1lyC6bBrp1bFX9cKmCbvE8m8cUEw0DxxGkhz'
access_token_secret = 'twBi4PhLwFfzsFyTvfDBY1jTgFIiSIN5QUSYPY8lUeZ8z'

In [5]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [6]:
now = datetime.today().now()
prev = now-timedelta(days=1)
prev=prev.strftime("%Y-%m-%d")

In [7]:
tweets_db = pd.DataFrame(columns = ["Original Content","Cleaned Content","Author", "Timestamp", "Retweet_count","Followers_count","Location", "Subjectivity", "Polarity"])

In [8]:
### FUNCTION FOR CLEANING THE TWEETS

def cleanTxt(text):
 text = re.sub('@[A-Za-z0–9]+', '', text) #Removing @mentions
 text = re.sub('#', '', text) # Removing '#' hash tag
 text = re.sub('RT[\s]+', '', text) # Removing RT
 text = re.sub('https?:\/\/\S+', '', text) # Removing hyperlink
 text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", text).split())

 return text

# Create a function to get the subjectivity
def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity

# Create a function to get the polarity
def getPolarity(text):
   return  TextBlob(text).sentiment.polarity

In [15]:
starttime = time.time()
while True: 
    tweets = tw.Cursor(api.search,
        q="Google" + " -filter:retweets",
        lang="en",
        since=prev).items(10)
    
    tweet_info = [[tweet.text,tweet.user.screen_name,tweet.created_at,tweet.retweet_count,tweet.user.followers_count,tweet.user.location] for tweet in tweets]
    tweet_df = pd.DataFrame(data=tweet_info, 
                    columns=["Original Content","Author", "Timestamp", "Retweet_count","Followers_count","Location"])
    
    ##Sentiment Analysis
    tweet_df['Cleaned Content'] = tweet_df['Original Content'].apply(cleanTxt)
    tweet_df['Subjectivity'] = tweet_df['Original Content'].apply(getSubjectivity)
    tweet_df['Polarity'] = tweet_df['Original Content'].apply(getPolarity)
    
    ##Moving averages and updating the tweet database
    tweets_db = pd.concat([tweets_db,tweet_df]).drop_duplicates().reset_index(drop=True)
    tweets_db['SMA_Polarity'] = tweets_db.Polarity.rolling(10, min_periods=1).mean()
    tweets_db['SMA_Subjectivity'] = tweets_db.Subjectivity.rolling(10, min_periods=1).mean()
    print("/nTweets updated./n")
    
    time.sleep(10.0 - ((time.time() - starttime) % 10.0))

/nTweets updated./n
                                    Original Content  \
0  @VaderReviews As much as I love that shot, my ...   
1  New York City to reopen middle schools as Chic...   
2  google this and it really just sent me down a ...   
3  @thegayprofiler @Annamaymay3 Do not leave me h...   
4  i'm doing comp title research &amp; google sug...   

                                     Cleaned Content         Author  \
0  As much as I love that shot my favorite ending...      jonbap426   
1  New York City to reopen middle schools as Chic...       aaa02930   
2  google this and it really just sent me down a ...         sllizs   
3                          3 Do not leave me hanging       ESDWales   
4  i m doing comp title research amp google sugge...  Grahamophones   

            Timestamp Retweet_count Followers_count               Location  \
0 2021-02-08 17:49:51             0              33                 Philly   
1 2021-02-08 17:49:50             0             175         

/nTweets updated./n
                                    Original Content  \
0  @VaderReviews As much as I love that shot, my ...   
1  New York City to reopen middle schools as Chic...   
2  google this and it really just sent me down a ...   
3  @thegayprofiler @Annamaymay3 Do not leave me h...   
4  i'm doing comp title research &amp; google sug...   

                                     Cleaned Content         Author  \
0  As much as I love that shot my favorite ending...      jonbap426   
1  New York City to reopen middle schools as Chic...       aaa02930   
2  google this and it really just sent me down a ...         sllizs   
3                          3 Do not leave me hanging       ESDWales   
4  i m doing comp title research amp google sugge...  Grahamophones   

            Timestamp Retweet_count Followers_count               Location  \
0 2021-02-08 17:49:51             0              33                 Philly   
1 2021-02-08 17:49:50             0             175         

/nTweets updated./n
                                    Original Content  \
0  @VaderReviews As much as I love that shot, my ...   
1  New York City to reopen middle schools as Chic...   
2  google this and it really just sent me down a ...   
3  @thegayprofiler @Annamaymay3 Do not leave me h...   
4  i'm doing comp title research &amp; google sug...   

                                     Cleaned Content         Author  \
0  As much as I love that shot my favorite ending...      jonbap426   
1  New York City to reopen middle schools as Chic...       aaa02930   
2  google this and it really just sent me down a ...         sllizs   
3                          3 Do not leave me hanging       ESDWales   
4  i m doing comp title research amp google sugge...  Grahamophones   

            Timestamp Retweet_count Followers_count               Location  \
0 2021-02-08 17:49:51             0              33                 Philly   
1 2021-02-08 17:49:50             0             175         

/nTweets updated./n
                                    Original Content  \
0  @VaderReviews As much as I love that shot, my ...   
1  New York City to reopen middle schools as Chic...   
2  google this and it really just sent me down a ...   
3  @thegayprofiler @Annamaymay3 Do not leave me h...   
4  i'm doing comp title research &amp; google sug...   

                                     Cleaned Content         Author  \
0  As much as I love that shot my favorite ending...      jonbap426   
1  New York City to reopen middle schools as Chic...       aaa02930   
2  google this and it really just sent me down a ...         sllizs   
3                          3 Do not leave me hanging       ESDWales   
4  i m doing comp title research amp google sugge...  Grahamophones   

            Timestamp Retweet_count Followers_count               Location  \
0 2021-02-08 17:49:51             0              33                 Philly   
1 2021-02-08 17:49:50             0             175         

/nTweets updated./n
                                    Original Content  \
0  @VaderReviews As much as I love that shot, my ...   
1  New York City to reopen middle schools as Chic...   
2  google this and it really just sent me down a ...   
3  @thegayprofiler @Annamaymay3 Do not leave me h...   
4  i'm doing comp title research &amp; google sug...   

                                     Cleaned Content         Author  \
0  As much as I love that shot my favorite ending...      jonbap426   
1  New York City to reopen middle schools as Chic...       aaa02930   
2  google this and it really just sent me down a ...         sllizs   
3                          3 Do not leave me hanging       ESDWales   
4  i m doing comp title research amp google sugge...  Grahamophones   

            Timestamp Retweet_count Followers_count               Location  \
0 2021-02-08 17:49:51             0              33                 Philly   
1 2021-02-08 17:49:50             0             175         

KeyboardInterrupt: 

In [16]:
tweets_db

,Original Content,Cleaned Content,Author,Timestamp,Retweet_count,Followers_count,Location,Subjectivity,Polarity,SMA_Polarity
0,"@VaderReviews As much as I love that shot, my ...",As much as I love that shot my favorite ending...,jonbap426,2021-02-08 17:49:51,0,33,Philly,0.800000,0.500000,0.500000
1,New York City to reopen middle schools as Chic...,New York City to reopen middle schools as Chic...,aaa02930,2021-02-08 17:49:50,0,175,,0.151515,0.045455,0.272727
2,google this and it really just sent me down a ...,google this and it really just sent me down a ...,sllizs,2021-02-08 17:49:50,0,0,"bae area, california",0.244444,0.022222,0.189226
3,@thegayprofiler @Annamaymay3 Do not leave me h...,3 Do not leave me hanging,ESDWales,2021-02-08 17:49:49,0,1082,"Wales, United Kingdom",0.000000,0.000000,0.141919
4,i'm doing comp title research &amp; google sug...,i m doing comp title research amp google sugge...,Grahamophones,2021-02-08 17:49:49,0,3374,New York City,0.000000,0.000000,0.113535
...,...,...,...,...,...,...,...,...,...,...
278,Also google says pansori is a form of storytel...,Also google says pansori is a form of storytel...,gorgxns,2021-02-08 18:14:55,0,135,,0.000000,0.000000,0.014470
279,@nakedwanderings @livelifenaked5 @ZoombeziBay ...,5 How well would poor French speaking,uSNaturistWater,2021-02-08 18:14:53,0,1200,USA,0.300000,-0.200000,-0.005530
280,@LeftOverPizza13 @DevDad08 @PatStaresAt what t...,13 8 what the fuck is google,Sparrow_VA,2021-02-08 18:14:53,0,257,,0.600000,-0.400000,-0.085530
281,@Cn25_98 @kratos_jr123 @Starboy_Saka @JoshPinh...,25 98 jr123 Saka ofc I was using Android for t...,UtdHas,2021-02-08 18:14:53,0,5,,0.000000,0.000000,-0.085530


,Original Content,Cleaned Content,Author,Timestamp,Retweet_count,Followers_count,Location,Subjectivity,Polarity,SMA_Polarity
0,"@VaderReviews As much as I love that shot, my ...",As much as I love that shot my favorite ending...,jonbap426,2021-02-08 17:49:51,0,33,Philly,0.800000,0.500000,0.500000
1,New York City to reopen middle schools as Chic...,New York City to reopen middle schools as Chic...,aaa02930,2021-02-08 17:49:50,0,175,,0.151515,0.045455,0.272727
2,google this and it really just sent me down a ...,google this and it really just sent me down a ...,sllizs,2021-02-08 17:49:50,0,0,"bae area, california",0.244444,0.022222,0.189226
3,@thegayprofiler @Annamaymay3 Do not leave me h...,3 Do not leave me hanging,ESDWales,2021-02-08 17:49:49,0,1082,"Wales, United Kingdom",0.000000,0.000000,0.141919
4,i'm doing comp title research &amp; google sug...,i m doing comp title research amp google sugge...,Grahamophones,2021-02-08 17:49:49,0,3374,New York City,0.000000,0.000000,0.113535


#### Twitter Sentiment Analysis

In [42]:
# Printing positive tweets 
sortedDF = tweets_db.sort_values(by=['Polarity'], ascending = False) #Sort the tweets

for tweet in sortedDF["Original Content"].head(10):
    print(tweet + "\n")

Google assumes that I’m 25 lol

Not Google Slides, but I love this!! https://t.co/91zqTHun6x

@samurai_badass ok google!

@ONGODBIEBS Fr💀
The baby lyrics than that "bdk" translation. Lmao I don't trust google anymore

@treasuremembers Ok, Google how to have honey voice like Yedam?

@DerekHarwell1 @johnstonmike34 @ELMLEnews @TaylorMali I was just looking through Google photos and I have so many p… https://t.co/rHSN7rKlfJ

@ChristopherRowe @scottedelman https://t.co/zq8bFrtw9p
https://t.co/km4kxaz3CC
Scottie for sure.

More Bible Verses: 
https://t.co/AgmFmYjr3M https://t.co/MUHmHBd9BR

More Bible Verses: 
https://t.co/AgmFmYjr3M https://t.co/y0hKnWpO0I

@cantguard_zayyd @eremox that's what Google says, but you can never be sure 👀



In [43]:
sortedDF.head()

,Original Content,Cleaned Content,Author,Timestamp,Retweet_count,Followers_count,Location,Subjectivity,Polarity
16,Google assumes that I’m 25 lol,Google assumes that I m 25 lol,_forevermaya_,2021-01-30 15:28:35,0,1174,"Raleigh, NC",0.7,0.80000
78,"Not Google Slides, but I love this!! https://t...",Not Google Slides but I love this,SlidesManiaSM,2021-01-30 15:30:25,0,36340,,0.6,0.78125
4,@samurai_badass ok google!,badass ok google,RaimeiL,2021-01-30 15:28:15,0,103,,0.5,0.62500
71,"@ONGODBIEBS Fr💀\nThe baby lyrics than that ""bd...",Fr The baby lyrics than that bdk translation L...,BizzIeFondue,2021-01-30 15:30:37,0,692,he/him,1.0,0.60000
61,"@treasuremembers Ok, Google how to have honey ...",Ok Google how to have honey voice like Yedam,harukyulovebot,2021-01-30 15:30:20,0,277,2311,0.5,0.50000
